<h1>Point_E Model Scanning Sample</h1>

<h3>Point_E Image To PointCloud<h3/>

In [ ]:
import tqdm
from AIScan import CloudSampler
from AIScan.Models import CreateModel
from Utils.Data import getPathImages, getPathNPBinaries
from Utils.Torch import UseBestTorchDevice

from point_e.util.point_cloud import PointCloud
from point_e.util.mesh import TriMesh
from point_e.util.pc_to_mesh import marching_cubes_mesh
from point_e.util.plotting import plot_point_cloud

srcPath = './data/duck'

In [ ]:
# Loading Sampler For Chosen Cloud

sampler = CloudSampler.CreateCloudSampler('base1B')

In [ ]:
# Loading Images From Provided Source Path (Provided In First Cell)

images = getPathImages(srcPath)
if (len(images) == 0):
    print("--__ No Images Found __--")
    exit()

In [ ]:
# Generate Samples from the Sampler, Using Provided Images

samples = None

for x in tqdm.tqdm(sampler.sample_batch_progressive(batch_size=1, model_kwargs=dict(images=images))):
    samples = x

del images

In [ ]:
# Save All Generated Samples To Cloud Files, 

figs = []

for i, cloud in enumerate(sampler.output_to_point_clouds(samples)):
    figs[i] = plot_point_cloud(cloud, grid_size=3, fixed_bounds=((-0.75, -0.75, -0.75),(0.75, 0.75, 0.75)))

    with open(f'{imageSrcPath}/cloud_{i}.npz', 'wb') as f:
        cloud.save(f)

del sampler
del samples

In [ ]:
# Clear All Variables from Memory
del sampler
del samples

del figs

<h3>Point_E Pointcloud To Mesh</h3>

In [ ]:
# Create SDF Generator Model

sdfModel = CreateModel('sdf', UseBestTorchDevice())

In [ ]:
# Genreating Meshes From Clouds, Previously Saved To NPZ Files

curCloud: PointCloud = None
curMesh = None
meshes = []

for cloudPath in getPathNPBinaries(srcPath):
    curCloud = PointCloud.load(cloudPath)

    curMesh = marching_cubes_mesh(
        pc = curCloud,
        model = sdfModel,
        batch_size = 4096,
        grid_size = 128,
        progress=True
    )

    meshes.append(curMesh)